# Imports

In [1]:
from pathlib import Path
import sys
from typing import Any
import warnings

import pandas as pd
import numpy as np

abs_path = Path().resolve()
sys.path.append(str(abs_path.parent / "modules"))

from files_funcs import *
from docx_funcs import *

warnings.filterwarnings("ignore")

# Config

In [2]:
main_dir = abs_path.parent.parent

config_path = abs_path.parent.parent / "config" / "config.yaml"
config = load_yaml(config_path)

data_dir = main_dir / "data"
uc_data_dir = data_dir / "raw_data" / "HMI"
ssts_data_dir = data_dir / "raw_data" / "SSTS"

columns = (
        config["ds_topology"]["main_columns"]
        + list(set(config["ds_topology"]["hmi_subheaders"].values()))
        + config["ds_topology"]["targets"]
)
columns = [col.lower().replace(" ", "_") for col in columns]

# Loading

In [3]:
hmi_files = [file.name for file in uc_data_dir.iterdir() if file.is_file()]
ssts_files = [file.name for file in ssts_data_dir.iterdir() if file.is_file()]

ids = set(
    [
        "".join(char for char in fname if char.isdigit())
        for fname in (
            hmi_files + ssts_files
    )
    ]
)

ds_rows = [
    create_ds_row(fid, uc_data_dir, ssts_data_dir, columns, config["ds_topology"]["hmi_subheaders"])
    for fid in ids
]

ds = pd.DataFrame(ds_rows, columns=columns)
ds

Warning! [Errno 2] No such file or directory: 'C:\\Users\\yytrb\\OneDrive\\Рабочий стол\\atom-compliance-ml\\data\\raw_data\\SSTS\\SSTS-26160.docx'


,id,case_name,full_uc_text,full_ssts_text,postconditions,other,preconditions,main_scenario,goal,alternative_scenario,differences,description,complience_level
0,8604,Source selection for Media output,[I-8604] Source selection for Media output\nD...,Switch music sources\nFunctional Description\n...,out_2/out_5 shows available media sources,,ATOM is on \nin_5. Smartphone is paired with t...,"User selects the ""Multimedia"" via in_5 or in_2...",,,None,None,None
1,26161,FM Radio Stations switching,[I-26161] FM Radio Stations switching\nDescri...,Automatic search\nFunctional Description\nSupp...,out_5/out_2 shows the station information \n24...,,User has selected FM Radio playback \nSmartpho...,User presses the forward/backward button via i...,,,None,None,None
2,30371,ERA Self-diagnosis,[I-30371] ERA Self-diagnosis\nActors:\nERA-Gl...,ERA Self-diagnosis\nFunctional Description\n\n...,,,,,,,None,None,None
3,8692,Emergency Service Communication (ERA-Glonass),[I-8692] Emergency Service Communication (ERA...,Manual dialing E-CALL\nFunctional Description\...,,,The vehicle is in the Drive State (PRND)\nThe ...,The ERA-Glonass initiates emergency service co...,,,None,None,None
4,25957,Mute/unmute the FM Radio playback,[I-25957] Mute/unmute the FM Radio playback\n...,Mute or pause function\nFunctional Description...,The user Mutes/Unmutes the FM Radio playback \...,,User is playing FM Radio,​​​User clicks on Mute/Unmute button via in_2/...,,,None,None,None
5,6583,Driver initiate a call through SWP,[I-6583] Driver initiate a call through SWP\n...,Make a call (B sample)\nFunctional Description...,"If the call is initiated successfully, the SWP...",,The IVI is on in_2 \nThe vehicle can be shifte...,The driver navigates to the 'Calls' option on ...,,The driver activates the voice assistant (in_2...,None,None,None
6,26160,Filtering the internet radio stations,[I-26160] Filtering the internet radio statio...,None,out_2/out_5 displays the internet radio statio...,,ATOM is connected to the internet \nUser has s...,out_2/out_5 displays the list of stations \nUs...,,,None,None,None
7,8800,Receiving Call Notifications,[I-8800] Receiving Call Notifications\nPrecon...,Receiving Call Notifications\nFunctional Descr...,"If the call is accepted, the interior ambient ...",,Entertainment system is ready. in_2 \nThe driv...,An incoming call is received on the driver's p...,,,None,None,None
8,31523,Adding Internet Radio to Favorites List,[I-31523] Adding Internet Radio to Favorites ...,Favorite Song operation\nFunctional Descriptio...,The selected internet radio station is added t...,,User in in the Internet Radio list,"The user press on the UI interface ""Add to Fav...",,,None,None,None
9,26771,Turn on and off hotspot via VA,[I-26771] Turn on and off hotspot via VA\n\nD...,\nTurn on and off hotspot\nFunctional Descript...,,,,The user asks in_24 Voice. Microphone to turn ...,,,None,None,None


# Target

In [4]:
marked_data = pd.read_excel(data_dir / "raw_data" / "train_data_markup.xlsx")
marked_data = marked_data[["Number", "Complience Level"]]
marked_data = marked_data.rename(columns={"Number": "id", "Complience Level": "complience_level"})
marked_data["target"] = marked_data["complience_level"].fillna("NA").map(config["target_dict"])
marked_data = marked_data.drop("complience_level", axis=1)
marked_data["id"] = marked_data["id"].astype(str)

ds = ds.merge(marked_data, on="id")
ds

,id,case_name,full_uc_text,full_ssts_text,postconditions,other,preconditions,main_scenario,goal,alternative_scenario,differences,description,complience_level,target
0,8604,Source selection for Media output,[I-8604] Source selection for Media output\nD...,Switch music sources\nFunctional Description\n...,out_2/out_5 shows available media sources,,ATOM is on \nin_5. Smartphone is paired with t...,"User selects the ""Multimedia"" via in_5 or in_2...",,,None,None,None,2
1,26161,FM Radio Stations switching,[I-26161] FM Radio Stations switching\nDescri...,Automatic search\nFunctional Description\nSupp...,out_5/out_2 shows the station information \n24...,,User has selected FM Radio playback \nSmartpho...,User presses the forward/backward button via i...,,,None,None,None,4
2,30371,ERA Self-diagnosis,[I-30371] ERA Self-diagnosis\nActors:\nERA-Gl...,ERA Self-diagnosis\nFunctional Description\n\n...,,,,,,,None,None,None,4
3,8692,Emergency Service Communication (ERA-Glonass),[I-8692] Emergency Service Communication (ERA...,Manual dialing E-CALL\nFunctional Description\...,,,The vehicle is in the Drive State (PRND)\nThe ...,The ERA-Glonass initiates emergency service co...,,,None,None,None,4
4,25957,Mute/unmute the FM Radio playback,[I-25957] Mute/unmute the FM Radio playback\n...,Mute or pause function\nFunctional Description...,The user Mutes/Unmutes the FM Radio playback \...,,User is playing FM Radio,​​​User clicks on Mute/Unmute button via in_2/...,,,None,None,None,3
5,6583,Driver initiate a call through SWP,[I-6583] Driver initiate a call through SWP\n...,Make a call (B sample)\nFunctional Description...,"If the call is initiated successfully, the SWP...",,The IVI is on in_2 \nThe vehicle can be shifte...,The driver navigates to the 'Calls' option on ...,,The driver activates the voice assistant (in_2...,None,None,None,2
6,26160,Filtering the internet radio stations,[I-26160] Filtering the internet radio statio...,None,out_2/out_5 displays the internet radio statio...,,ATOM is connected to the internet \nUser has s...,out_2/out_5 displays the list of stations \nUs...,,,None,None,None,1
7,8800,Receiving Call Notifications,[I-8800] Receiving Call Notifications\nPrecon...,Receiving Call Notifications\nFunctional Descr...,"If the call is accepted, the interior ambient ...",,Entertainment system is ready. in_2 \nThe driv...,An incoming call is received on the driver's p...,,,None,None,None,4
8,31523,Adding Internet Radio to Favorites List,[I-31523] Adding Internet Radio to Favorites ...,Favorite Song operation\nFunctional Descriptio...,The selected internet radio station is added t...,,User in in the Internet Radio list,"The user press on the UI interface ""Add to Fav...",,,None,None,None,4
9,26771,Turn on and off hotspot via VA,[I-26771] Turn on and off hotspot via VA\n\nD...,\nTurn on and off hotspot\nFunctional Descript...,,,,The user asks in_24 Voice. Microphone to turn ...,,,None,None,None,4


# Save

In [5]:
ds.to_csv(data_dir / "ds" / "ds.unl", sep="|", encoding="utf-8", index=False)